In [25]:
import pandas as pd
import numpy as np

In [26]:
#Last Trading Day
# Trading will cease at the close of business two UK Business Days prior to
# the first calendar day of the delivery month, quarter, season, or calendar.

ice_ttf= pd.read_excel('curves/data/ICE_TTF_FUT.xlsx')  
ice_ttf.columns = ["tradedate"] + [f'M{i:02}' for i in range(1, 61)]
ice_ttf['tradedate'] = pd.to_datetime(ice_ttf['tradedate'])

In [27]:
# Reset the index to make 'trade_date' a column again for melting
ice_ttf_long = pd.melt(ice_ttf, id_vars=['tradedate'], var_name='tenor', value_name='price')

In [28]:
# Convert 'month_number' to string if it's not already, then extract the last two characters and convert to integer
ice_ttf_long['offset'] = ice_ttf_long['tenor'].astype(str).str[-2:].astype(int)
ice_ttf_long

,tradedate,tenor,price,offset
0,2013-01-21,M01,26.040,1
1,2013-01-22,M01,26.340,1
2,2013-01-23,M01,26.330,1
3,2013-01-24,M01,25.850,1
4,2013-01-25,M01,26.100,1
...,...,...,...,...
160495,2023-05-24,M60,25.887,60
160496,2023-05-25,M60,24.801,60
160497,2023-05-26,M60,25.061,60
160498,2023-05-29,M60,25.036,60


In [37]:
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay
from pandas.tseries.holiday import AbstractHolidayCalendar, DateOffset, Holiday, next_monday, next_monday_or_tuesday, GoodFriday, EasterMonday, nearest_workday, MO

# Define a custom UK business day calendar that accounts for UK public holidays
class UKBusinessCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('New Year’s Day', month=1, day=1, observance=nearest_workday),
        GoodFriday,
        EasterMonday,
        Holiday('Early May Bank Holiday', month=5, day=1, offset=DateOffset(weekday=MO(1))),
        Holiday('Spring Bank Holiday', month=5, day=31, offset=DateOffset(weekday=MO(-1))),
        Holiday('Summer Bank Holiday', month=8, day=31, offset=DateOffset(weekday=MO(-1))),
        Holiday('Christmas Day', month=12, day=25, observance=next_monday),
        Holiday('Boxing Day', month=12, day=26, observance=next_monday_or_tuesday)
    ]
# Create a custom business day offset using the UK business calendar
uk_bd = CustomBusinessDay(calendar=UKBusinessCalendar())

# Function to calculate the last trading day which is two UK business days before the first of next month
def calculate_last_trading_day(date):
    first_day_next_month = pd.Timestamp(date.year + int(date.month == 12), date.month % 12 + 1, 1)
    last_trading_day = first_day_next_month - 2 * uk_bd
    return last_trading_day

# Function to calculate the first day of the tenor month
def calculate_tenor_month_start(trade_date):
    # Calculate the last trading day for the given trade_date's month
    last_trading_day_current_month = calculate_last_trading_day(trade_date)
    
    # If the trade_date is after the last trading day, the tenor month is the next month +1
    if trade_date > last_trading_day_current_month:
        tenor_month_start = pd.Timestamp(trade_date.year + int(trade_date.month == 12), trade_date.month % 12 + 1, 1)
    else:
        # Otherwise, the tenor month is the current month +1
        tenor_month_start = pd.Timestamp(trade_date.year, trade_date.month, 1)
    
    return tenor_month_start

# Apply the function to calculate the first day of the tenor month for each trade_date
ice_ttf_long['last_business_day'] = ice_ttf_long['tradedate'].apply(calculate_last_trading_day)

ice_ttf_long[['last_business_day']].drop_duplicates()

# Now 'tenor_month_start' is a column with the first day of the tenor month

,last_business_day
0,2013-01-30
9,2013-02-27
29,2013-03-27
49,2013-04-29
71,2013-05-30
...,...
2569,2023-01-30
2591,2023-02-27
2611,2023-03-30
2634,2023-04-27


In [30]:
df_melted

,tradedate,tenor,price
0,2013-01-21,M01,26.040
1,2013-01-22,M01,26.340
2,2013-01-23,M01,26.330
3,2013-01-24,M01,25.850
4,2013-01-25,M01,26.100
...,...,...,...
160495,2023-05-24,M60,25.887
160496,2023-05-25,M60,24.801
160497,2023-05-26,M60,25.061
160498,2023-05-29,M60,25.036


In [31]:
prices = raw

logret = np.log(prices.shift(1)/prices)

NameError: name 'raw' is not defined